<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/code_clones_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My\ Drive/code-clones

/content/gdrive/My Drive/code-clones


In [0]:
import xmlextractor
import metrics
import os
import pandas as pd
import numpy as np
import pickle

In [0]:
with open('csv/duplicates.pkl', 'rb') as f:
  duplicates = pickle.load(f)

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

def resample(X, y):
  rus = RandomUnderSampler(random_state=23)
  return rus.fit_resample(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
dup = []
for duplicate in duplicates:
    text = '\n\n'.join(duplicate.fragments)
    dup.append({'text': text, 'exp': duplicate.exp})

In [0]:
df_tfidf = pd.DataFrame(dup)
X = np.array(df_tfidf['text'])
y = np.array(df_tfidf['exp'])
X_resampled, y_resampled = resample(X.reshape(-1, 1), y.reshape(-1, 1))
X_resampled = X_resampled.reshape(1, -1)[0]
y_resampled = y_resampled.reshape(1, -1)[0]

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(ngram_range=(1,1))
X_count = count_vec.fit_transform(X_resampled)
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_count)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_resampled, random_state=23)

In [9]:
%cd /
!git clone https://github.com/hyperopt/hyperopt-sklearn
%cd hyperopt-sklearn
!pip install -e .

/
Cloning into 'hyperopt-sklearn'...
remote: Enumerating objects: 1176, done.
remote: Total 1176 (delta 0), reused 0 (delta 0), pack-reused 1176
Receiving objects: 100% (1176/1176), 1.99 MiB | 18.88 MiB/s, done.
Resolving deltas: 100% (715/715), done.
/hyperopt-sklearn
Obtaining file:///hyperopt-sklearn
  Running setup.py develop for hpsklearn


In [10]:
from hpsklearn import HyperoptEstimator, any_sparse_classifier, tfidf
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [0]:
def custom_loss(y_target, y_prediction):
  k_fn = 1
  k_fp = 10
  n = len(y_target)
  (fnr, fpr) = metrics.calc_fnr_fpr(y_target, y_prediction)
  return k_fn * fnr + k_fp * fpr

In [12]:
estim = HyperoptEstimator(classifier=any_sparse_classifier('clf'),
                          preprocessing=[],
                          algo=tpe.suggest,
                          trial_timeout=300,
                          loss_fn=custom_loss,
                          seed=23)
estim.fit(X_train, y_train)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



100%|██████████| 1/1 [00:00<00:00, 16.27it/s, best loss: 1.1000613873542051]


In [17]:
y_pred = estim.predict(X_test)
metrics.print_fnr_fpr(y_test, y_pred)
metrics.print_metrics(y_test, y_pred)


FNR: 
0.19373848987108655

FPR: 
0.09907918968692449

accuracy: 
0.7071823204419889

precision: 
0.761101243339254

recall: 
0.6196673897324656


In [14]:
estim.best_model()

{'ex_preprocs': (),
 'learner': SGDClassifier(alpha=0.06118428364662971, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0003970052556398526,
               fit_intercept=True, l1_ratio=0.5022724095962902,
               learning_rate='constant', loss='log', max_iter=158684350.0,
               n_iter_no_change=5, n_jobs=1, penalty='l2',
               power_t=0.907051629874976, random_state=0, shuffle=True,
               tol=0.006308140398304822, validation_fraction=0.1, verbose=False,
               warm_start=False),
 'preprocs': ()}

In [0]:
y_prob = [y[1] for y in estim.best_model()['learner'].predict_proba(X_test)]

In [16]:
metrics.print_roc_auc(y_test, y_prob)


roc auc: 
0.8001469473812206
